#### This notebook demonstrates the use of InFoRM algorithms to mitigate bias for LINE embedding
InFoRM includes 3 algorithms, namely debiasing the input graph, debiasing the mining model and debiasing the mining result. We will show how to run all 3 algorithms for LINE in this notebook.

### Get vanilla embedding matrix first

In [1]:
# load necessary packages
import pickle
import load_graph
import utils

import numpy as np
import networkx as nx
import sklearn.preprocessing as skpp

import torch
import torch.nn as nn

In [2]:
# from vanilla_line import *
class LINE:
    def __init__(self, dimension=128, ratio=3200, negative=5, batch_size=1000, init_lr=0.025, seed=None):
        self.dimension = dimension
        self.ratio = ratio
        self.negative = negative
        self.batch_size = batch_size
        self.init_lr = init_lr
        if seed is not None:
            np.random.seed(seed)

    def train(self, graph):
        self.graph = graph
        self.is_directed = nx.is_directed(self.graph)
        self.num_node = graph.number_of_nodes()
        self.num_sampling_edge = self.ratio * self.num_node

        node2id = dict([(node, vid) for vid, node in enumerate(graph.nodes())])
        self.edges = [[node2id[e[0]], node2id[e[1]]] for e in self.graph.edges()]
        self.edges_prob = np.asarray([graph[u][v].get("weight", 1.0) for u, v in graph.edges()])
        self.edges_prob /= np.sum(self.edges_prob)
        self.edges_table, self.edges_prob = alias_setup(self.edges_prob)

        degree_weight = np.asarray([0] * self.num_node)
        for u, v in graph.edges():
            degree_weight[node2id[u]] += graph[u][v].get("weight", 1.0)
            if not self.is_directed:
                degree_weight[node2id[v]] += graph[u][v].get("weight", 1.0)
        self.node_prob = np.power(degree_weight, 0.75)
        self.node_prob /= np.sum(self.node_prob)
        self.node_table, self.node_prob = alias_setup(self.node_prob)

        self.emb_vertex = (np.random.random((self.num_node, self.dimension)) - 0.5) / self.dimension
        self.fair_emb_vertex = self.emb_vertex.copy()
        self._train_line()
        self.embeddings = skpp.normalize(self.emb_vertex, "l2")
        return self.embeddings

    def _update(self, vec_u, vec_v, vec_error, label):
        # update vetex embedding and vec_error
        f = 1 / (1 + np.exp(-np.sum(vec_u * vec_v, axis=1)))
        g = (self.lr * (label - f)).reshape((len(label), 1))
        vec_error += g * vec_v
        vec_v += g * vec_u

    def _train_line(self):
        self.lr = self.init_lr
        batch_size = self.batch_size
        num_batch = int(self.num_sampling_edge / batch_size)
        epoch_iter = range(num_batch)
        for b in epoch_iter:
            # if b % self.batch_size == 0:
            #     self.lr = self.init_lr * max((1 - b * 1.0 / num_batch), 0.0001)
            self.lr = self.init_lr * max((1 - b * 1.0 / num_batch), 0.0001)
            u, v = [0] * batch_size, [0] * batch_size
            for i in range(batch_size):
                edge_id = alias_draw(self.edges_table, self.edges_prob)
                u[i], v[i] = self.edges[edge_id]
                if not self.is_directed and np.random.rand() > 0.5:
                    v[i], u[i] = self.edges[edge_id]

            vec_error = np.zeros((batch_size, self.dimension))
            label, target = np.asarray([1 for i in range(batch_size)]), np.asarray(v)
            for j in range(self.negative + 1):
                if j != 0:
                    label = np.asarray([0 for i in range(batch_size)])
                    for i in range(batch_size):
                        target[i] = alias_draw(self.node_table, self.node_prob)
                self._update(
                    self.emb_vertex[u], self.emb_vertex[target], vec_error, label
                )
            self.emb_vertex[u] += vec_error

def alias_setup(probs):
    """
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    """
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K * prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q


def alias_draw(J, q):
    """
    Draw sample from a non-uniform discrete distribution using alias sampling.
    """
    K = len(J)

    kk = int(np.floor(np.random.rand() * K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [3]:
def vanilla(name, ratio, seed):
    try:
        with open('result/line/vanilla.pickle', 'rb') as f:
            edict = pickle.load(f)
    except:
        edict = dict()

    data = load_graph.read_pickle(name)
    adj_train = data['adjacency_train']
    graph = nx.from_scipy_sparse_matrix(adj_train, create_using=nx.Graph(), edge_attribute='weight')

    # train
    model = LINE(ratio=ratio, seed=seed)
    edict[name] = model.train(graph)

    with open('result/line/vanilla.pickle'.format(ratio, seed), 'wb') as f:
        pickle.dump(edict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
vanilla(name='ppi', ratio=3200, seed=0)

### Let's debias the input graph

In [5]:
# load debias model
from method.debias_graph import DebiasGraph

In [6]:
def debias_input_graph(name, alpha=0., lr=0., metric=None):
    # load dataset
    data = load_graph.read_pickle(name)
    init_adj = data['adjacency_train']

    # build similarity matrix
    sim = utils.filter_similarity_matrix(utils.get_similarity_matrix(init_adj, metric=metric), sigma=0.75)

    # debias LINE
    FairGraph = DebiasGraph()
    adj = FairGraph.line(init_adj, sim, alpha, maxiter=100, lr=lr, tol=1e-6)
    graph = nx.from_scipy_sparse_matrix(adj, create_using=nx.Graph(), edge_attribute='weight')
    model = LINE(ratio=3200, seed=0)
    embs = model.train(graph)

    print('dataset: {}\tmetric: {} similarity'.format(name, metric))
    print('Finished!')

    return embs

In [7]:
# jaccard index
result = dict()
result['ppi'] = debias_input_graph('ppi', alpha=10, lr=0.025, metric='jaccard')
with open('result/line/graph/jaccard.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

# cosine similarity    
result = dict()
result['ppi'] = debias_input_graph('ppi', alpha=10, lr=0.025, metric='cosine')
with open('result/line/graph/cosine.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

dataset: ppi	metric: jaccard similarity
Finished!
dataset: ppi	metric: cosine similarity
Finished!


### Let's debias the mining model

In [8]:
# load debias model
from method.debias_model import DebiasModel

In [9]:
def debias_mining_model(name, alpha=0., metric=None):
    # load dataset
    data = load_graph.read_pickle(name)
    adj_train = data['adjacency_train']
    graph = nx.from_scipy_sparse_matrix(adj_train, create_using=nx.Graph(), edge_attribute='weight')

    # build similarity matrix
    sim = utils.filter_similarity_matrix(utils.get_similarity_matrix(adj_train, metric=metric), sigma=0.75)

    # debias LINE
    FairModel = DebiasModel()
    embs = FairModel.line(
        graph, sim, alpha,
        dimension=128, ratio=3200, negative=5,
        init_lr=0.025, batch_size=1000, seed=0
    )

    print('dataset: {}\tmetric: {} similarity'.format(name, metric))
    print('Finished!')

    return embs

In [10]:
alpha = 0.5
# jaccard index
result = dict()
result['ppi'] = debias_mining_model(name='ppi', alpha=alpha, metric='jaccard')
with open('result/line/model/jaccard.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

# cosine similarity    
result = dict()
result['ppi'] = debias_mining_model(name='ppi', alpha=alpha, metric='cosine')
with open('result/line/model/cosine.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

dataset: ppi	metric: jaccard similarity
Finished!
dataset: ppi	metric: cosine similarity
Finished!


### Let's debias the mining result

In [11]:
# load debias model
from method.debias_result import DebiasResult

In [12]:
def debias_mining_result(name, vanilla, alpha=0., metric=None):
    # vanilla embeddings
    embs = vanilla[name]

    # load dataset
    data = load_graph.read_pickle(name)
    adj_train = data['adjacency_train']

    # build similarity matrix
    sim = utils.filter_similarity_matrix(utils.get_similarity_matrix(adj_train, metric=metric), sigma=0.75)

    # debias LINE
    FairResult = DebiasResult()
    embs = FairResult.fit(embs, sim, alpha)

    print('dataset: {}\tmetric: {} similarity'.format(name, metric))
    print()

    return embs

In [13]:
alpha = 0.5
with open('result/line/vanilla.pickle', 'rb') as f:
    vanilla = pickle.load(f)

# jaccard index
result = dict()
result['ppi'] = debias_mining_result(name='ppi', vanilla=vanilla, alpha=alpha, metric='jaccard')
with open('result/line/result/jaccard.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

# cosine similarity    
result = dict()
result['ppi'] = debias_mining_result(name='ppi', vanilla=vanilla, alpha=alpha, metric='cosine')
with open('result/line/result/cosine.pickle', 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)

dataset: ppi	metric: jaccard similarity

dataset: ppi	metric: cosine similarity



### Now, let's see how much we debiased and how good debiased results are

In [14]:
# load evaluation functions
from evaluate.line import *

In [15]:
evaluate(name='ppi', metric='jaccard', task='graph')
evaluate(name='ppi', metric='cosine', task='graph')
evaluate(name='ppi', metric='jaccard', task='model')
evaluate(name='ppi', metric='cosine', task='model')
evaluate(name='ppi', metric='jaccard', task='result')
evaluate(name='ppi', metric='cosine', task='result')

{'dataset': 'ppi', 'metric': 'jaccard similarity', 'task': 'debias the input graph', 'diff': 0.6740087760748081, 'roc-auc': (0.6818451361318293, 0.6784420320651908), 'f1': (0.6178672863413375, 0.6201910663568293), 'bias': {'train': [0.020624031681280353], 'validation': [0.03629812978226654], 'test': [0.015421255425152935]}}
{'dataset': 'ppi', 'metric': 'cosine similarity', 'task': 'debias the input graph', 'diff': 0.6993078976931937, 'roc-auc': (0.6818451361318293, 0.6855929039010265), 'f1': (0.6178672863413375, 0.6207074619158275), 'bias': {'train': [0.012206033740946975], 'validation': [0.03451971868807613], 'test': [0.02550044009603536]}}
{'dataset': 'ppi', 'metric': 'jaccard similarity', 'task': 'debias the mining model', 'diff': 0.23823116757757387, 'roc-auc': (0.6818451361318293, 0.7154187140657258), 'f1': (0.6178672863413375, 0.6423960753937517), 'bias': {'train': [0.05852995194383237], 'validation': [0.25737305180892533], 'test': [0.2644720912112759]}}
{'dataset': 'ppi', 'metri